### --- Day 9: Rope Bridge ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/9

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 9);
WriteLines(inputLines);

Total lines: 2000
Max line length: 4

L 1
R 2
L 1
U 2
D 1


In [4]:
string[] testInputLines = 
[
    "R 4",
    "U 4",
    "L 3",
    "D 1",
    "R 4",
    "D 1",
    "L 5",
    "R 2",
];

In [5]:
Dictionary<char, Point> Directions = new()
{
    { 'R', Right },
    { 'L', Left },
    { 'U', Up },
    { 'D', Down },
};

The tricky part about this one is that the tail doesn't always move after two head movements. It does for purely horizontal or vertical movements, but in the case where the head turns a corner, eg `across`, `up`, `up`, the tail doesn't move until the third head movement.

What is true however is that the tail is always "attached" to the head. So I think we can check that whenever we have a horizontal or vertical difference of `2`, we know where we need to place the tail.

In [6]:
string[] demoMatrix = [
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
    "..........................",
];

Point demoStart = (11, 15);

In [ ]:
int RunSim(string[] inputLines, int knotCount = 2, Func<Point, Point, Point> chaseFunc = null)
{
    // Spoiler alert: knot count and chase function change in part 2
    chaseFunc ??= Chase;

    Point[] allKnots = new Point[knotCount];
    HashSet<Point> tailVisited = [];

    foreach (var line in inputLines)
    {
        var direction = Directions[line[0]];
        var steps = int.Parse(line[2..]);

        tailVisited.UnionWith(MoveAndChase(direction, steps));
    }

    return tailVisited.Count;

    IEnumerable<Point> MoveAndChase(Point direction, int steps)
    {
        foreach (var _ in Enumerable.Range(0, steps))
        {
            allKnots[0]+= direction;

            foreach (var i in Enumerable.Range(1, allKnots.Length - 1))
            {
                var head = allKnots[i-1];
                var tail = allKnots[i];
                var newTail = chaseFunc(head, tail);
                allKnots[i] = newTail;
            }

            var finalTail = allKnots[^1];
            yield return finalTail;
        }
    }
}

Point Chase(Point head, Point tail)
{
    // See comments above for how this handles the literal corner case :)

    return (head - tail) switch 
    {
        (> 1, _) => (head.X + Left.X, head.Y),
        (< -1, _) => (head.X + Right.X, head.Y),
        (_, > 1) => (head.X, head.Y + Up.Y),
        (_, < -1) => (head.X, head.Y + Down.Y),
        _ => tail
    };


In [8]:
// So, there are 13 positions the tail visited at least once.

var testAnswer = RunSim(testInputLines);
Console.WriteLine(testAnswer);

In [9]:
// Simulate your complete hypothetical series of motions. How many positions
// does the tail of the rope visit at least once?

var part1Answer = RunSim(inputLines);
Console.WriteLine(part1Answer);

In [10]:
// 6181 is correct!
Ensure(6181, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/9

Ok, so in part two, fundamentally we can progressively treat each knot pair as the head / tail, but there is one complication. Suppose we have the following:

```
.....
...0.
..1..
.2...
.....
```

In this case, if the head moves up, point `1` will move across and underneath point `0`. This now leaves a gap of 2 rows and 2 columns between `1` and `2`, eg:

```
...0.
...1.
.....
.2...
.....
```

This was a case that couldn't happen in part one, and we need to account for it in part two. In this case, we need to place point `2` on the diagonal of `1` eg:

```
...0.
...1.
..2..
.....
.....
```

Therefore we need to update our `Chase` function to accomodate this new case.

In [12]:
Point ChasePt2(Point head, Point tail)
{
    return (head - tail) switch 
    {
        (> 1, > 1) => (head.X + Left.X, head.Y + Up.Y),
        (> 1, < -1) => (head.X + Left.X, head.Y + Down.Y),
        (< -1, > 1) => (head.X + Right.X, head.Y + Up.Y),
        (< -1, < -1) => (head.X + Right.X, head.Y + Down.Y),

        _ => Chase(head, tail)
    };
}

In [13]:
// Now, the tail (9) visits 36 positions (including s) at least once:

string[] part2TestInputLines = 
[
    "R 5",
    "U 8",
    "L 8",
    "D 3",
    "R 17",
    "D 10",
    "L 25",
    "U 20",
];

var part2TestAnswer = RunSim(part2TestInputLines, knotCount: 10, chaseFunc: ChasePt2);
Console.WriteLine(part2TestAnswer);

In [14]:
// Simulate your complete series of motions on a larger rope with ten knots. How
// many positions does the tail of the rope visit at least once?

var part2Answer = RunSim(inputLines, knotCount: 10, chaseFunc: ChasePt2);
Console.WriteLine(part2Answer);

In [15]:
// 2386 is correct!
Ensure(2386, part2Answer);